In [1]:
!pip install transformers
import torch
import json
import json
from pathlib import Path
from torch.utils.data import DataLoader
from transformers import AutoTokenizer
import pickle
from transformers import AdamW,BertForQuestionAnswering
from tqdm.notebook import tqdm
import string, re

     ---------------------------------------- 0.0/6.8 MB ? eta -:--:--
     - -------------------------------------- 0.3/6.8 MB 5.9 MB/s eta 0:00:02
     ------ --------------------------------- 1.2/6.8 MB 12.3 MB/s eta 0:00:01
     --------------------- ------------------ 3.7/6.8 MB 26.3 MB/s eta 0:00:01
     ---------------------------------------  6.7/6.8 MB 35.6 MB/s eta 0:00:01
     ---------------------------------------- 6.8/6.8 MB 33.3 MB/s eta 0:00:00
     ---------------------------------------- 0.0/3.5 MB ? eta -:--:--
     ------------------------- -------------- 2.2/3.5 MB 71.7 MB/s eta 0:00:01
     ---------------------------------------- 3.5/3.5 MB 44.4 MB/s eta 0:00:00
     ---------------------------------------- 0.0/200.1 kB ? eta -:--:--
     ------------------------------------- 200.1/200.1 kB 12.7 MB/s eta 0:00:00


c:\anaconda\anaconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\anaconda\anaconda3\lib\site-packages\numpy\.libs\libopenblas.EL2C6PLE4ZYW3ECEVIV3OXXGRN2NRFM2.gfortran-win_amd64.dll
c:\anaconda\anaconda3\lib\site-packages\numpy\.libs\libopenblas64__v0.3.21-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [2]:
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
model = torch.load("savedModels/saved_model.pkl",map_location=torch.device('cpu'))

c:\anaconda\anaconda3\lib\site-packages\huggingface_hub\file_download.py:133: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Rohit Irvisetty\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [3]:
def ans(context,query):
  answer_start = torch.argmax(model(**tokenizer.encode_plus(query, context, return_tensors='pt'))[0]) 
  answer_end = torch.argmax(model(**tokenizer.encode_plus(query, context, return_tensors='pt'))[1]) + 1 
  return tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(tokenizer.encode_plus(query, context, return_tensors='pt')['input_ids'][0][answer_start:answer_end]))

In [4]:
def normalize(input):
  def remove_punc(text):
    exclude = set(string.punctuation)
    return "".join(ch for ch in text if ch not in exclude)

  return " ".join((re.sub(re.compile(r"\b(a|an|the)\b", re.UNICODE), " ", remove_punc(input.lower()))).split())

In [5]:
def f1_score(prediction, truth):
  pred_tokens = normalize(prediction).split()
  truth_tokens = normalize(truth).split()
  
  if len(pred_tokens) == 0 or len(truth_tokens) == 0:
    return int(pred_tokens == truth_tokens)
    
  if len(set(pred_tokens) & set(truth_tokens)) == 0:
    return 0
  
  return 2 * (len(set(pred_tokens) & set(truth_tokens)) / len(pred_tokens) * len(set(pred_tokens) & set(truth_tokens)) / len(truth_tokens)) / (len(set(pred_tokens) & set(truth_tokens)) / len(pred_tokens) + len(set(pred_tokens) & set(truth_tokens)) / len(truth_tokens))

In [6]:
def Predict_Ans(context,query,answer):
  prediction = ans(context,query)
  f1 = f1_score(prediction, answer)

  print(f"Prediction: {prediction}")
  print(f"F1: {f1}")

In [8]:
context = "Hi! My name is Alexa and I am 21 years old. I used to live in Peristeri of Athens, but now I moved on in Kaisariani of Athens."

queries = ["How old is Alexa?",
           "Where does Alexa live now?",
           "Where Alexa used to live?"
          ]
answers = ["21",
           "Kaisariani of Athens",
           "Peristeri of Athens"
          ]

for q,a in zip(queries,answers):
  print(f"Question: {q}")
  print(f"Ground truth: {a}")
  Predict_Ans(context,q,a)

Question: How old is Alexa?
Ground truth: 21
Prediction: 21
F1: 1.0
Question: Where does Alexa live now?
Ground truth: Kaisariani of Athens
Prediction: kaisariani
F1: 0.5
Question: Where Alexa used to live?
Ground truth: Peristeri of Athens
Prediction: peristeri
F1: 0.5
